In [15]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from functools import partial
import time

In [9]:
# load data
with open('train.pkl', 'rb') as f:
    data = pickle.load(f)

In [45]:
# image prepearation for dominace
import cv2
import os
X = []
y = []
for i, row in data.iterrows():
    if i > 10000: # only use first 10000 images
        break
    try:
        x1, y1, x2, y2 = row['bbox'][0], row['bbox'][1], row['bbox'][2], row['bbox'][3]
        img = cv2.imread(row['paths'])[y1:y2, x1:x2] # crop image
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to grayscale
        img = cv2.resize(img, (224, 224)) # resize to 224x224
        img = img / 255.0 # normalize
        img = np.array(img) 
        X.append(img)
        y.append(row['dominance'])
    except:
        continue
    
X = np.asarray(X)
y = np.asarray(y)

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: extra compressed data
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
Corrupt JPEG data: 50 extraneous bytes before marker 0xd9


In [19]:
# callbacks
root_logdir = os.path.join(os.curdir, "tb_logs")
def get_run_logdir(param_str):    
    run_id = str(int(time.time())) + param_str
    return os.path.join(root_logdir, run_id)
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [46]:
# simle CNN model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
from functools import partial
DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=2, activation='relu', padding="SAME")

model = keras.models.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape = (224,224,1)),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=10, activation='softmax'),
])

run_logdir = get_run_logdir("_simpleCNN_" + "dominance")
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
es = tf.keras.callbacks.EarlyStopping(min_delta=1.0, patience=10, verbose=1)
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=5, validation_split=0.1, callbacks=[tensorboard_cb, es])

score = model.evaluate(X_test, y_test)
X_new = X_test[:10] 
y_pred = model.predict(X_new)

Epoch 1/5


2022-07-20 16:19:57.270315: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


223/223 [==============================] - ETA: 0s - loss: 1.8776 - accuracy: 0.2809

2022-07-20 16:22:01.418642: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


223/223 [==============================] - 130s 575ms/step - loss: 1.8776 - accuracy: 0.2809 - val_loss: 1.8016 - val_accuracy: 0.3363
Epoch 2/5
223/223 [==============================] - 127s 569ms/step - loss: 1.8080 - accuracy: 0.2963 - val_loss: 1.7650 - val_accuracy: 0.3363
Epoch 3/5
223/223 [==============================] - 126s 567ms/step - loss: 1.7844 - accuracy: 0.3021 - val_loss: 1.7553 - val_accuracy: 0.3363
Epoch 4/5
223/223 [==============================] - 127s 568ms/step - loss: 1.7772 - accuracy: 0.3031 - val_loss: 1.7474 - val_accuracy: 0.3363
Epoch 5/5
62/62 [==============================] - 13s 213ms/step - loss: 1.7733 - accuracy: 0.3114


2022-07-20 16:30:47.442906: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [30]:
data

,bbox,paths,discrete_emotion,valance,arousal,dominance,first_emotion
0,"[86, 58, 564, 628]",./emotic/mscoco/images/COCO_val2014_0000005622...,"[Disconnection, Doubt/Confusion]",5,3,9,12
1,"[485, 149, 605, 473]",./emotic/mscoco/images/COCO_train2014_00000028...,[Anticipation],6,4,7,3
2,"[305, 92, 461, 465]",./emotic/mscoco/images/COCO_val2014_0000005581...,"[Engagement, Excitement, Happiness]",7,8,8,4
3,"[221, 63, 448, 372]",./emotic/mscoco/images/COCO_train2014_00000036...,"[Aversion, Pleasure]",8,9,8,17
4,"[44, 143, 150, 288]",./emotic/mscoco/images/COCO_train2014_00000021...,"[Confidence, Excitement]",7,9,10,5
...,...,...,...,...,...,...,...
17072,"[189, 194, 323, 438]",./emotic/mscoco/images/COCO_val2014_0000002037...,"[Anticipation, Engagement]",6,5,3,3
17073,"[214, 48, 340, 326]",./emotic/mscoco/images/COCO_train2014_00000017...,[Confidence],7,8,7,5
17074,"[166, 35, 341, 401]",./emotic/mscoco/images/COCO_val2014_0000005140...,"[Anticipation, Engagement, Excitement]",6,2,10,3
17075,"[245, 227, 293, 340]",./emotic/framesdb/images/frame_k7fb824vh221kl3...,[Engagement],5,5,6,4


In [34]:
import cv2
import os
X = []
y = []
for i, row in data.iterrows():
    if i > 10: # only use first 10000 images
        break
    try:
        x1, y1, x2, y2 = row['bbox'][0], row['bbox'][1], row['bbox'][2], row['bbox'][3]
        img = cv2.imread(row['paths'])[y1:y2, x1:x2] # crop image
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to grayscale
        img = cv2.resize(img, (224, 224)) # resize to 224x224
        img = img / 255.0 # normalize
        img = np.array(img) 
        X.append(img)
        y.append(row['first_emotion'])
        print(row['first_emotion'])
    except:
        continue
    
X = np.asarray(X)
y = np.asarray(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

12
3
4
17
5
3
3
3
4
13
24


In [47]:

%tensorboard --logdir tb_logs

UsageError: Line magic function `%tensorboard` not found.


In [2]:
# training xception model with keras
import tensorflow as tf
import keras
from keras.applications.xception import Xception
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img

model = Xception(weights='imagenet', include_top=False, input_shape=(224,224,3))



ValueError: The input must have 3 channels; Received `input_shape=(224, 224, 1)`